# <span style="color:darkblue"> Lecture 21 - Text Data  </span>

<font size = "5">

In  this lecture we will work with text data

<font size = "5">

Import Libraries

In [1]:
import pandas as pd

<font size = "5">

Import Data

- Congressional bills in the United States

In [2]:
bills_actions = pd.read_csv("data_raw/bills_actions.csv")
bills_actions.dtypes

Congress        int64
bill_number     int64
bill_type      object
action         object
main_action    object
category       object
member_id       int64
dtype: object

# <span style="color:darkblue"> II. Basic Text Operations </span>

<font size = "5">

Count Frequency

In [3]:
bills_actions["category"]

0         amendment
1         amendment
2         amendment
3       senate bill
4       senate bill
           ...     
3298      amendment
3299      amendment
3300      amendment
3301      amendment
3302      amendment
Name: category, Length: 3303, dtype: object

In [4]:
bills_actions["category"].value_counts()

amendment                       1529
house bill                       902
senate bill                      514
house resolution                 234
senate resolution                 60
house joint resolution            22
house concurrent resolution       20
senate concurrent resolution      14
senate joint resolution            8
Name: category, dtype: int64

<font size = "5">

Subset text categories

In [5]:
# For this analysis we are only interested in bills. With ".query()" ...
#     - We select all entries in the column called "category" 
#       which have values contain in "list_categories"
#     - "in" is used to test whether a word belongs to a list
#     - @ is the syntax to reference "global" variables that
#       are defined in the global environment

list_categories = ["house bill","senate bill"]
bills = bills_actions.query('category in @list_categories')

# Verify that the code worked:
bills["category"].value_counts()


house bill     902
senate bill    514
Name: category, dtype: int64

<font size = "5">

Data manipulation with sentences

In [6]:
# How many bills mention the word Senator?
bool_contains = bills["action"].str.contains("Senator")

print(bool_contains.mean())

0.3199152542372881


In [7]:
bills[bills["action"].str.contains("Senator")]

,Congress,bill_number,bill_type,action,main_action,category,member_id
3,116,1199,s,"Committee on Health, Education, Labor, and Pen...",senate committee/subcommittee actions,senate bill,1561
4,116,1208,s,Committee on the Judiciary. Reported by Senato...,senate committee/subcommittee actions,senate bill,1580
5,116,1231,s,Committee on the Judiciary. Reported by Senato...,senate committee/subcommittee actions,senate bill,1580
6,116,1228,s,"Committee on Commerce, Science, and Transporta...",senate committee/subcommittee actions,senate bill,1002
7,116,123,s,Committee on Veterans' Affairs. Reported by Se...,senate committee/subcommittee actions,senate bill,1490
...,...,...,...,...,...,...,...
2944,116,617,hr,Committee on Energy and Natural Resources. Rep...,senate committee/subcommittee actions,house bill,1581
3081,116,762,hr,Committee on Energy and Natural Resources. Rep...,senate committee/subcommittee actions,house bill,1581
3142,116,828,hr,Committee on Homeland Security and Governmenta...,senate committee/subcommittee actions,house bill,1701
3150,116,829,hr,Committee on Homeland Security and Governmenta...,senate committee/subcommittee actions,house bill,1701


In [9]:
# How to replace the word "Senator" with "Custom Title"
bills["action_custom"] = bills["action"].str.replace("Senator","Sen.")

/var/folders/3g/kh8280_j3b5515nc6yp293880000gn/T/ipykernel_33406/372703810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bills["action_custom"] = bills["action"].str.replace("Senator","Sen.")


<font size = "5">

Try it yourself!

- Obtain a new dataset called "resolutions" <br>
 which subsets rows contain the "category" values:

 ``` ["house resolution","senate resolution"] ```

In [10]:
# Write your own code

list_resolution_names = ["house resolution","senate resolution"]
resolutions    = bills_actions.query(" category in @list_resolution_names")

resolutions



,Congress,bill_number,bill_type,action,main_action,category,member_id
485,116,123,sres,Committee on Foreign Relations. Reported by Se...,senate committee/subcommittee actions,senate resolution,505
486,116,135,sres,Committee on Foreign Relations. Reported by Se...,senate committee/subcommittee actions,senate resolution,505
487,116,142,sres,Committee on Foreign Relations. Reported by Se...,senate committee/subcommittee actions,senate resolution,505
488,116,152,sres,Committee on Foreign Relations. Reported by Se...,senate committee/subcommittee actions,senate resolution,505
489,116,183,sres,Committee on Foreign Relations. Reported by Se...,senate committee/subcommittee actions,senate resolution,505
...,...,...,...,...,...,...,...
1085,116,603,hres,Mr. Hoyer moved to table the measure.,house floor actions,house resolution,1065
1086,116,603,hres,QUESTION OF THE PRIVILEGES OF THE HOUSE - The ...,house floor actions,house resolution,1560
1087,116,647,hres,Mr. Hoyer moved to table the measure.,house floor actions,house resolution,1065
1088,116,770,hres,Mr. Hoyer moved to table the measure.,house floor actions,house resolution,1065


# <span style="color:darkblue"> III. Regular Expressions </span>

<font size = "5">

Regular expressions enable advanced searching <br>
for string data.

In [ ]:
# A regular expression (or regex) is a sequence of characters that defines a search pattern,
# usually for finding, extracting, or replacing parts of text.

dataset = pd.read_csv("data_raw/bills_actions.csv")

senate_bills = dataset.query('category == "senate bill"')
amendments = dataset.query('category == "amendment"')

In [12]:
to_reconsider = dataset[dataset['action'].str.contains('to reconsider')]

In [13]:
display(amendments)

,Congress,bill_number,bill_type,action,main_action,category,member_id
0,116,1029,s,S.Amdt.1274 Amendment SA 1274 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
1,116,1031,s,S.Amdt.2698 Amendment SA 2698 proposed by Sena...,senate amendment proposed (on the floor),amendment,675
2,116,1160,s,S.Amdt.2659 Amendment SA 2659 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
8,116,1253,s,S.Amdt.2424 Amendment SA 2424 proposed by Sena...,senate amendment proposed (on the floor),amendment,297
11,116,1309,s,S.Amdt.1275 Amendment SA 1275 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
...,...,...,...,...,...,...,...
3298,116,9,hr,H.Amdt.172 Amendment (A004) offered by Ms. Kus...,house amendment offered,amendment,36
3299,116,9,hr,H.Amdt.171 Amendment (A003) offered by Ms. Hou...,house amendment offered,amendment,186
3300,116,9,hr,H.Amdt.170 Amendment (A002) offered by Ms. Oma...,house amendment offered,amendment,477
3301,116,9,hr,POSTPONED PROCEEDINGS - At the conclusion of d...,other house amendment actions,amendment,393


<font size = "5">

Search word

In [ ]:
# We use the ".str.findall()" subfunction
# The argument is an expression

# "Amdt\.....\D" just means the rows that will be selected will have:
# "Amdt"
# "\." = just a "."   -> since a "." means something in regex, we need to have a "\"" before the "." to indicate that we mean the literal "."
# "...."  -> 4 dots means any 4 characters after the actual "."
# "\D"  -> matches 1 non-digit character (so basically any character other than 0-9)

# code goes thru every observation and looks at the action column -> if there is a string that matches these requirements,
# it is extracted thru the findall function
amendments["action"].str.findall("Amdt\.....\D")

0       [Amdt.1274 ]
1       [Amdt.2698 ]
2       [Amdt.2659 ]
8       [Amdt.2424 ]
11      [Amdt.1275 ]
            ...     
3298    [Amdt.172 A]
3299    [Amdt.171 A]
3300    [Amdt.170 A]
3301              []
3302    [Amdt.169 A]
Name: action, Length: 1529, dtype: object

<font size = "5">

Wildcards

$\quad$ <img src="figures/wildcards_regex1.png" alt="drawing" width="300"/>

In [15]:
# Get digits after string
example1 = amendments["action"].str.findall("Amdt\..")

# Get any character before string
example2 = amendments["action"].str.findall(".Amdt\.")

# Get two characters before string
example3 = amendments["action"].str.findall("..Amdt\....")

display(example1)
display(example2)
display(example3)

0       [Amdt.1]
1       [Amdt.2]
2       [Amdt.2]
8       [Amdt.2]
11      [Amdt.1]
          ...   
3298    [Amdt.1]
3299    [Amdt.1]
3300    [Amdt.1]
3301          []
3302    [Amdt.1]
Name: action, Length: 1529, dtype: object

0       [.Amdt.]
1       [.Amdt.]
2       [.Amdt.]
8       [.Amdt.]
11      [.Amdt.]
          ...   
3298    [.Amdt.]
3299    [.Amdt.]
3300    [.Amdt.]
3301          []
3302    [.Amdt.]
Name: action, Length: 1529, dtype: object

0       [S.Amdt.127]
1       [S.Amdt.269]
2       [S.Amdt.265]
8       [S.Amdt.242]
11      [S.Amdt.127]
            ...     
3298    [H.Amdt.172]
3299    [H.Amdt.171]
3300    [H.Amdt.170]
3301              []
3302    [H.Amdt.169]
Name: action, Length: 1529, dtype: object

<font size = "5">

Wildcards + Quantifiers

$\quad$ <img src="figures/wildcards_regex2.png" alt="drawing" width="300"/>

In [ ]:
# "*" is associated wth the preceeding character
# ".*Amdt\S*" means:
# ".*"   -> any character and all those like it (aka everything preceeding Amdt will be extracted)
# "Amdt"
# "\S*"  -> non-whitespace character and all those like it (aka will keep extracting if the following character is a non-whitespace character; will stop when string reaches a character that is NOT non-whitespace)

amendments["action"].str.findall(".*Amdt\S*")

0       [S.Amdt.1274]
1       [S.Amdt.2698]
2       [S.Amdt.2659]
8       [S.Amdt.2424]
11      [S.Amdt.1275]
            ...      
3298     [H.Amdt.172]
3299     [H.Amdt.171]
3300     [H.Amdt.170]
3301               []
3302     [H.Amdt.169]
Name: action, Length: 1529, dtype: object

In [22]:
# Get all consecutive digits after string
example4 = amendments["action"].str.findall("Amdt\.\d*")

# Get all consecutive characters before string
example5 = amendments["action"].str.findall(".*Amdt\.")

display(example4)
display(example5)


0       [Amdt.1274]
1       [Amdt.2698]
2       [Amdt.2659]
8       [Amdt.2424]
11      [Amdt.1275]
           ...     
3298     [Amdt.172]
3299     [Amdt.171]
3300     [Amdt.170]
3301             []
3302     [Amdt.169]
Name: action, Length: 1529, dtype: object

0       [S.Amdt.]
1       [S.Amdt.]
2       [S.Amdt.]
8       [S.Amdt.]
11      [S.Amdt.]
          ...    
3298    [H.Amdt.]
3299    [H.Amdt.]
3300    [H.Amdt.]
3301           []
3302    [H.Amdt.]
Name: action, Length: 1529, dtype: object

<font size = "5">

Try it yourself

- Practice using the ```senate_bills``` dataset
- Use ```.str.findall()``` to find the word "Senator"
- Use the regular expression ```"Senator \S``` to extract <br>
 the the first letter of senator
- Use ```*``` to extract senator names

In [19]:
# Write your own code

display(senate_bills["action"].str.findall("Senator \S"))
display(senate_bills["action"].str.findall("Senator \S*"))

3      [Senator A]
4      [Senator G]
5      [Senator G]
6      [Senator W]
7      [Senator M]
          ...     
795    [Senator J]
796             []
797    [Senator H]
798             []
799    [Senator G]
Name: action, Length: 514, dtype: object

3      [Senator Alexander]
4         [Senator Graham]
5         [Senator Graham]
6         [Senator Wicker]
7          [Senator Moran]
              ...         
795      [Senator Johnson]
796                     []
797       [Senator Hoeven]
798                     []
799       [Senator Graham]
Name: action, Length: 514, dtype: object